In [1]:
import json
from glob import glob
from unidecode import unidecode

In [2]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    'idiom in',
    'in English',
    'in Malay',
    # 'language model'
]

In [9]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian',
    'mistral',
]

In [16]:
instructions = []
rejected = [
    'math', 
    'multiturn', 
    'dangerous', 
    'starter', 
    'conversation', 
    'rag', 
    'audio',
    'ahli-parlimen',
    'disagree'
]

files = glob('/home/husein/ssd3/soalan-augmentation/mixtral*.jsonl')
files = [f for f in files if all([r not in f for r in rejected])]
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l_lower = l.lower()
            l = json.loads(l)
            if any([s in l_lower for s in skip]):
                # print(l)
                continue
            
            q = l.get('question', '').strip()
            if not len(q):
                continue
            if not q.endswith('?'):
                continue
            if len(q) < 200 and '?' not in q:
                # print(q)
                continue
            a = l.get('answer_ms')
            if a is None:
                continue
            a = a.strip()
            if not len(a):
                continue
            if len(a) < len(q):
                continue
            if len(a) < 100:
                continue
                
            d = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            instructions.append(d)
            
len(instructions)

253189

In [17]:
glob('/home/husein/ssd3/soalan-augmentation/*math*.jsonl')

['/home/husein/ssd3/soalan-augmentation/mixtral-conversation-math-stupid.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-math.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-simple-math.jsonl']

In [19]:
!head -n 1 /home/husein/ssd3/soalan-augmentation/mixtral-simple-math.jsonl

{"question": "How many factors are there for 12? (1, 2, 3, 4, 6, 12)", "question_ms": "Berapakah bilangan faktor untuk 12? (1, 2, 3, 4, 6, 12)", "answer": "Yes, you are correct. The factors of 12 are 1, 2, 3, 4, 6, and 12. A factor is a whole number that can be divided evenly into another number. In this case, each of these numbers divides evenly into 12 with no remainder.", "answer_ms": "Ya, anda betul. Faktor 12 ialah 1, 2, 3, 4, 6, dan 12. Faktor ialah nombor keseluruhan yang boleh dibahagikan sama rata kepada nombor lain. Dalam kes ini, setiap nombor ini dibahagikan sama rata kepada 12 tanpa baki."}


In [20]:
files = glob('/home/husein/ssd3/soalan-augmentation/*math*.jsonl')
files = [f for f in files if 'conversation' not in f]
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            q = l.get('question', '').strip()
            if not len(q):
                continue
            a = l.get('answer_ms')
            if a is None:
                continue
            a = a.strip()
            if not len(a):
                continue
            if len(a) < len(q):
                continue
            if len(a) < 100:
                continue
                
            d = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            instructions.append(d)
            
len(instructions)

265871

In [21]:
with open('prepared-mixtral-malaysian-general-qa.jsonl', 'w') as fopen:
    for d in instructions:
        fopen.write(f'{json.dumps(d)}\n')

In [25]:
files = glob('/home/husein/ssd3/soalan-augmentation/*conversation*.jsonl')
files.extend(glob('/home/husein/ssd3/soalan-augmentation/*multiturn*.jsonl'))
files = [f for f in files if 'rag' not in f]
files

['/home/husein/ssd3/soalan-augmentation/mixtral-conversation-math-stupid.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-conversation-stupid.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-conversation-badthings.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-critics-politician-multiturn.jsonl',
 '/home/husein/ssd3/soalan-augmentation/mixtral-critics-malaysia-multiturn.jsonl']

In [26]:
from tqdm import tqdm

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

files = glob('/home/husein/ssd3/soalan-augmentation/*conversation*.jsonl')
files.extend(glob('/home/husein/ssd3/soalan-augmentation/*multiturn*.jsonl'))
files = [f for f in files if 'rag' not in f]
instructions = []
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                s = f"{role}: {current_text}"
                    
                inputs.append(s)

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
                
            if len(inputs) < 3:
                continue

            data = '\n'.join(inputs).strip()
            if not len(data):
                continue
                
            d = {
                'prompt_input': None,
                'input': data,
                'output': None,
            }
            instructions.append(d)
            
len(instructions)

48163it [00:07, 6507.13it/s]
60384it [00:11, 5191.44it/s]
57798it [00:09, 6326.60it/s]
135770it [01:30, 1499.40it/s]
103242it [00:52, 1975.68it/s]


345475

In [28]:
with open('prepared-mixtral-malaysian-multiturn.jsonl', 'w') as fopen:
    for d in instructions:
        fopen.write(f'{json.dumps(d)}\n')

In [40]:
rags = [
    '/home/husein/ssd3/soalan-augmentation/mixtral-rag-question-disagree.jsonl',
    '/home/husein/ssd3/soalan-augmentation/mixtral-audio-instruction.jsonl',
]

instructions = []
for f in rags:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            context = l['context']
            l = l['chat']
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                if no == 0:
                    current_text = f"{context}\n{current_text}"
                
                s = f"{role}: {current_text}"
                    
                inputs.append(s)

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
                
            if len(inputs) < 3:
                continue

            data = '\n'.join(inputs).strip()
            if not len(data):
                continue
                
            d = {
                'prompt_input': None,
                'input': data,
                'output': None,
            }
            instructions.append(d)

339627it [01:27, 3891.31it/s]
293752it [01:22, 3549.50it/s]


In [41]:
with open('prepared-mixtral-malaysian-rag.jsonl', 'w') as fopen:
    for d in instructions:
        fopen.write(f'{json.dumps(d)}\n')

In [43]:
!ls -lh prepared-mixtral-malaysian-rag.jsonl

-rw-r--r-- 1 husein husein 2.8G Jan  14 15:28 prepared-mixtral-malaysian-rag.jsonl


In [42]:
!head -n 1 prepared-mixtral-malaysian-rag.jsonl

{"prompt_input": null, "input": "<manusia>: Ingat lagi dengan permainan The Sim yang menarik minat ramai pemain diluar sana suatu ketika dahulu? Kini, pembuatnya kembali dengan judul permainan yang baru \u2013 Spore. Spore dijangka dikeluarkan pada 7 September 2008 di mana para peminat juga boleh memuat turun permainan ini dari laman Electronic Arts pada 7 September.\n\n\nAntaramuka Permainan Spore\n\nMelalui Spore, pemain boleh mengawal evolusi sesuatu spesis daripada permulaannya \u2013 unisel, pembesarannya, dan sehingga ia menjadi suatu makhluk yang berkebolehan dan sosial. Sekiranya anda sambung bermain selepas itu, anda boleh mengawal spesis anda sehingga berjaya ke angkasa lepas, disamping dapat berhubung dengan spesis lain dalam galaksi.\n\nPermainan ini dipecahkan kepada beberapa fasa, dimana disetiap fasa, cara permainan itu berubah-ubah. Sekiranya ketika bermain, spesis anda diserang atau dihancurkan sekali gus, anda akan kembali bermain pada level anda sebelum dihancurkan, 